In [29]:
import time
import pandas as pd
import numpy as np
import math
from math import cos, asin, sqrt
import gc
import os.path


pd.set_option('display.max_columns', None)

In [2]:
start = time.time()
#########################controlar la excepcion si no esta el fichero creado-----------------------------------------
vuelos = pd.read_csv('vuelos.csv', sep=',',low_memory=False, )
vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

end = time.time()
print(end - start)

5.27900004387


In [3]:
start = time.time()

if vuelos.empty:
    #hay que tener cuidado de no machacar nada.
    vuelos = pd.read_csv('vuelosDeparted.csv', sep=',',low_memory=False, )
    vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

    vuelos["TMIN_o"]= ''
    vuelos["TMIN_d"]= ''
    vuelos["TMAX_o"]= ''
    vuelos["TMAX_d"]= ''
    vuelos["TAVG_o"]= ''
    vuelos["TAVG_d"]= ''
    vuelos["SNOW_o"]= ''
    vuelos["SNOW_d"]= ''
    vuelos["PRCP_o"]= ''
    vuelos["PRCP_d"]= ''
    vuelos["SNWD_o"]= ''
    vuelos["SNWD_d"]= ''
    vuelos["ACMC_o"]= ''
    vuelos["ACMC_d"]= ''
    vuelos["ACSC_o"]= ''
    vuelos["ACSC_d"]= ''
    vuelos["AWDR_o"]= ''
    vuelos["AWDR_d"]= ''
    vuelos["AWND_o"]= ''
    vuelos["AWND_d"]= ''
    vuelos["EVAP_o"]= ''
    vuelos["EVAP_d"]= ''
    vuelos["FRTH_o"]= ''
    vuelos["FRTH_d"]= ''
    vuelos["TSUN_o"]= ''
    vuelos["TSUN_d"]= ''
    vuelos["WDMV_o"]= ''
    vuelos["WDMV_d"]= ''


    vuelos.to_csv('vuelos.csv', sep=',', index=False)

end = time.time()
print(end - start)

0.0


In [4]:
start = time.time()
##Leemos el fichero de estaciones de noaa

stationstxt = ""
with open("stations.txt") as input:
    stationstxt = input.read()
    
#Extract the data from file
stations2 = stationstxt.split("\n")
#Remove last line
stations2 = stations2[:-1]
stations2 = map(lambda line: [line[0:11],float(line[13:20]),float(line[22:30]),line[41:71]], stations2)

end = time.time()
print(end - start)

0.453000068665


# Leemos datos del tiempo de noaa

In [5]:
start = time.time()

#weather2012 = pd.read_csv("2012.csv",header=None)
#weather2012.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]

#weather2011 = pd.read_csv("2011.csv",header=None)
#weather2011.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2014 = pd.read_csv("2014.csv",header=None)
#weather2014.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2015 = pd.read_csv("2015.csv",header=None)
#weather2015.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2017 = pd.read_csv("2017.csv",header=None)
#weather2017.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
weather2016 = pd.read_csv("2016.csv",header=None)
weather2016.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]


end = time.time()
print(end - start)

36.5880000591


In [6]:
start = time.time()

##nos quedamos con las latitudes de las estaciones.
stationsdf = pd.DataFrame(stations2)
stationsdf.columns = ["id","lat","lng","name"]
lats = stationsdf[['lat','lng']]

#funciones para calcular la distancia entre los aeropuertos y estacion metereologica mas cercana
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v[0],v[1],p[0],p[1]))


#para que la fecha no de problemas al comparar (quizas no este bien del todo)
def tratarFecha(fecha):
    fecha = str(fecha).split(' ')[0]
    fecha = fecha.replace('-','')
    fecha = int(fecha)
    return fecha
end = time.time()
print(end - start)

0.304000139236


In [9]:
#con esto se ve que la distancia de la funcion es en km
#distance(40.380028, -3.702393, 41.376809, 2.147827)

504.0789260202627

In [7]:
start = time.time()
weatherdf = weather2016
del weather2016
del weatherdf['Value3']
del weatherdf['Value4']
del weatherdf['Value5']
del weatherdf['Value2']
weather = weatherdf ##creamos bck
end = time.time()
print(end - start)

2.7619998455


In [8]:
def obtenerEstacion2(row):
    lat = row['board_lat']
    lon = row['board_lon']
    coor = [lat,lon]
    EMC= closest(np.asarray(lats), coor)
    ##obtenemos el id de la estacion de board
    aux = stationsdf[stationsdf['lat']==EMC[0]]
    aux = stationsdf[stationsdf['lng']==EMC[1]]
    EMC = aux['id']
    s =EMC.to_string()
    stationid = str(s)
    row['stationid'] = stationid.split()[1]
    return row

def obtenerEstacion3(row):
    lat = row['off_lat']
    lon = row['off_lon']
    coor = [lat,lon]
    EMC= closest(np.asarray(lats), coor)
    ##obtenemos el id de la estacion de board
    aux = stationsdf[stationsdf['lat']==EMC[0]]
    aux = stationsdf[stationsdf['lng']==EMC[1]]
    EMC = aux['id']
    s =EMC.to_string()
    stationid = str(s)
    row['stationid'] = stationid.split()[1]
    return row

def aplicarEstacionesOrigen(row):
    row['board_stationid']= coordenadasOrigen[(coordenadasOrigen.board_lat == row['board_lat'])&(coordenadasOrigen.board_lon == row['board_lon'])]['stationid'].values[0]
    return row

def aplicarEstacionesDestino(row):
    row['off_stationid']= coordenadasDestino[(coordenadasDestino.off_lat == row['off_lat'])&(coordenadasDestino.off_lon == row['off_lon'])]['stationid'].values[0]
    return row

In [ ]:
#aplicamos y obtenemos las estaciones (para mejorar el procesado)

########################################################no ejecutar siempre--cntrolar que si existen ya no haga este paso..
coordenadasOrigen = vuelos[['board_lat','board_lon']]
coordenadasDestino = vuelos[['off_lat','off_lon']]
coordenadasOrigen = coordenadasOrigen.drop_duplicates(subset=['board_lat', 'board_lon'])
coordenadasDestino = coordenadasDestino.drop_duplicates(subset=['off_lat', 'off_lon'])
coordenadasOrigen = coordenadasOrigen.apply(lambda x:obtenerEstacion2(x),axis = 1)
coordenadasDestino = coordenadasDestino.apply(lambda x:obtenerEstacion3(x),axis = 1)
vuelos = vuelos.apply(lambda x: aplicarEstacionesOrigen(x),axis = 1)
vuelos = vuelos.apply(lambda x: aplicarEstacionesDestino(x),axis = 1)

In [9]:
def tratar2(row):
    
    board_station = row['board_stationid']
    off_station = row['off_stationid']
    fecha = row['actual_time_of_departure']
    fecha_d = row['actual_time_of_arrival'] 
    ##################################
    #primero tratamos las fechas
    fecha = tratarFecha(fecha)
    fecha_d = tratarFecha(fecha_d)
    #########################################################################################################
    data_board = weatherdf[(weatherdf["id"]==board_station)&(weatherdf['date']==fecha)].sort_values("date")
    data_off   = weatherdf[(weatherdf["id"]==off_station)&(weatherdf['date']==fecha_d)].sort_values("date")
    #####################################################
    data_board["Value1"]=data_board["Value1"]/10
    data_board = data_board.reset_index()
    data_off["Value1"]=data_off["Value1"]/10
    data_off = data_off.reset_index()
        #######################################################
    T1 = data_board[data_board['type']=='TMIN']['Value1']
    T2 = data_off[data_off['type']=='TMIN']['Value1']
    T3 = data_board[data_board['type']=='TMAX']['Value1']
    T4 = data_off[data_off['type']=='TMAX']['Value1']
    T5 = data_board[data_board['type']=='TAVG']['Value1']
    T6 = data_off[data_off['type']=='TAVG']['Value1']
    T7 = data_board[data_board['type']=='SNOW']['Value1']
    T8 = data_off[data_off['type']=='SNOW']['Value1']
    T9 = data_board[data_board['type']=='PRCP']['Value1']
    T10 = data_off[data_off['type']=='PRCP']['Value1']
    T11 = data_board[data_board['type']=='SNWD']['Value1']
    T12 = data_off[data_off['type']=='SNWD']['Value1']
    T13 = data_board[data_board['type']=='ACMC']['Value1']
    T14 = data_off[data_off['type']=='ACMC']['Value1']
    T17 = data_board[data_board['type']=='ACSC']['Value1']
    T18 = data_off[data_off['type']=='ACSC']['Value1']
    T21 = data_board[data_board['type']=='AWDR']['Value1']
    T22 = data_off[data_off['type']=='AWDR']['Value1']
    T23 = data_board[data_board['type']=='AWND']['Value1']
    T24 = data_off[data_off['type']=='AWND']['Value1']
    T39 = data_board[data_board['type']=='EVAP']['Value1']
    T40 = data_off[data_off['type']=='EVAP']['Value1']
    T47 = data_board[data_board['type']=='FRTH']['Value1']
    T48 = data_off[data_off['type']=='FRTH']['Value1']
    T75 = data_board[data_board['type']=='TSUN']['Value1']
    T76 = data_off[data_off['type']=='TSUN']['Value1']
    T89 = data_board[data_board['type']=='WDMV']['Value1']
    T90 = data_off[data_off['type']=='WDMV']['Value1']
    
    
    ##############################
    ##tratamos los valores nulos aplicando un valor nulo
    if T1.empty:
        T1 = 9999
    if T2.empty:
        T2 = 9999
    if T3.empty:
        T3 = 9999
    if T4.empty:
        T4 = 9999
    if T5.empty:
        T5 = 9999
    if T6.empty:
        T6 = 9999
    if T7.empty:
        T7 = 9999
    if T8.empty:
        T8 = 9999
    if T9.empty:
        T9 = 9999
    if T10.empty:
        T10 = 9999
    if T11.empty:
        T11 = 9999
    if T12.empty:
        T12 = 9999
    if T13.empty:
        T13 = 9999
    if T14.empty:
        T14 = 9999
    if T17.empty:
        T17 = 9999
    if T18.empty:
        T18 = 9999
    if T21.empty:
        T21 = 9999
    if T22.empty:
        T22 = 9999
    if T23.empty:
        T23 = 9999
    if T24.empty:
        T24 = 9999
    if T39.empty:
        T39 = 9999
    if T40.empty:
        T40 = 9999
    if T47.empty:
        T47 = 9999
    if T48.empty:
        T48 = 9999
    if T75.empty:
        T75 = 9999
    if T76.empty:
        T76 = 9999
    if T89.empty:
        T89 = 9999
    if T90.empty:
        T90 = 9999
      
        
    #actualizamos la fila
    row['TMIN_o']= float(T1)
    row['TMIN_d']= float(T2)
    row['TMAX_o']= float(T3)
    row['TMAX_d']= float(T4)
    row['TAVG_o']= float(T5)
    row['TAVG_d']= float(T6)
    row['SNOW_o']= float(T7)
    row['SNOW_d']= float(T8)
    row['PRCP_o']= float(T9)
    row['PRCP_d']= float(T10)
    row['SNWD_o']= float(T11)
    row['SNWD_d']= float(T12)
    row['ACMC_o']= float(T13)
    row['ACMC_d']= float(T14)
    row['ACSC_o']= float(T17)
    row['ACSC_d']= float(T18)
    row['AWDR_o']= float(T21)
    row['AWDR_d']= float(T22)
    row['AWND_o']= float(T23)
    row['AWND_d']= float(T24)
    row['EVAP_o']= float(T39)
    row['EVAP_d']= float(T40)
    row['FRTH_o']= float(T47)
    row['FRTH_d']= float(T48)
    row['TSUN_o']= float(T75)
    row['TSUN_d']= float(T76)
    row['WDMV_o']= float(T89)
    row['WDMV_d']= float(T90)


    return row

def functionWeatherDay(year, month, day):
    u = int(month+day)
    d = int(year*10000)
    weatherdf = weather##restauramos el bck
    weatherdf = weatherdf[weatherdf['date']-d==u]
    return weatherdf

def functionPlus(year, month, day):    
    start = time.time()
    vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
     
    x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==year]
    x = x[x['mesSalida']==month]
    x = x[x['diaSalida']==day]


    x = x.apply(lambda x:tratar2(x),axis = 1)
    #para escribir ya
    #variable auxiliar

    pd.options.mode.chained_assignment = None  # default='warn'
    vuelosSinTratar['TMIN_o'] = x.set_index(['index'])['TMIN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_o']).values
    vuelosSinTratar['TMIN_d'] = x.set_index(['index'])['TMIN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_d']).values
    vuelosSinTratar['TMAX_o'] = x.set_index(['index'])['TMAX_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_o']).values
    vuelosSinTratar['TMAX_d'] = x.set_index(['index'])['TMAX_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_d']).values
    vuelosSinTratar['TAVG_o'] = x.set_index(['index'])['TAVG_o'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_o']).values
    vuelosSinTratar['TAVG_d'] = x.set_index(['index'])['TAVG_d'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_d']).values
    vuelosSinTratar['SNOW_o'] = x.set_index(['index'])['SNOW_o'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_o']).values
    vuelosSinTratar['SNOW_d'] = x.set_index(['index'])['SNOW_d'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_d']).values
    vuelosSinTratar['PRCP_o'] = x.set_index(['index'])['PRCP_o'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_o']).values
    vuelosSinTratar['PRCP_d'] = x.set_index(['index'])['PRCP_d'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_d']).values
    vuelosSinTratar['SNWD_o'] = x.set_index(['index'])['SNWD_o'].combine_first(vuelosSinTratar.set_index(['index'])['SNWD_o']).values
    vuelosSinTratar['SNWD_d'] = x.set_index(['index'])['SNWD_d'].combine_first(vuelosSinTratar.set_index(['index'])['SNWD_d']).values
    vuelosSinTratar['ACMC_o'] = x.set_index(['index'])['ACMC_o'].combine_first(vuelosSinTratar.set_index(['index'])['ACMC_o']).values
    vuelosSinTratar['ACMC_d'] = x.set_index(['index'])['ACMC_d'].combine_first(vuelosSinTratar.set_index(['index'])['ACMC_d']).values
    vuelosSinTratar['ACSC_o'] = x.set_index(['index'])['ACSC_o'].combine_first(vuelosSinTratar.set_index(['index'])['ACSC_o']).values
    vuelosSinTratar['ACSC_d'] = x.set_index(['index'])['ACSC_d'].combine_first(vuelosSinTratar.set_index(['index'])['ACSC_d']).values
    vuelosSinTratar['AWDR_o'] = x.set_index(['index'])['AWDR_o'].combine_first(vuelosSinTratar.set_index(['index'])['AWDR_o']).values
    vuelosSinTratar['AWDR_d'] = x.set_index(['index'])['AWDR_d'].combine_first(vuelosSinTratar.set_index(['index'])['AWDR_d']).values
    vuelosSinTratar['AWND_o'] = x.set_index(['index'])['AWND_o'].combine_first(vuelosSinTratar.set_index(['index'])['AWND_o']).values
    vuelosSinTratar['AWND_d'] = x.set_index(['index'])['AWND_d'].combine_first(vuelosSinTratar.set_index(['index'])['AWND_d']).values
    vuelosSinTratar['EVAP_o'] = x.set_index(['index'])['EVAP_o'].combine_first(vuelosSinTratar.set_index(['index'])['EVAP_o']).values
    vuelosSinTratar['EVAP_d'] = x.set_index(['index'])['EVAP_d'].combine_first(vuelosSinTratar.set_index(['index'])['EVAP_d']).values
    vuelosSinTratar['FRTH_o'] = x.set_index(['index'])['FRTH_o'].combine_first(vuelosSinTratar.set_index(['index'])['FRTH_o']).values
    vuelosSinTratar['FRTH_d'] = x.set_index(['index'])['FRTH_d'].combine_first(vuelosSinTratar.set_index(['index'])['FRTH_d']).values
    vuelosSinTratar['TSUN_o'] = x.set_index(['index'])['TSUN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TSUN_o']).values
    vuelosSinTratar['TSUN_d'] = x.set_index(['index'])['TSUN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TSUN_d']).values
    vuelosSinTratar['WDMV_o'] = x.set_index(['index'])['WDMV_o'].combine_first(vuelosSinTratar.set_index(['index'])['WDMV_o']).values
    vuelosSinTratar['WDMV_d'] = x.set_index(['index'])['WDMV_d'].combine_first(vuelosSinTratar.set_index(['index'])['WDMV_d']).values


    vuelos['TMIN_o'] = vuelosSinTratar.set_index(['index'])['TMIN_o'].combine_first(vuelos.set_index(['index'])['TMIN_o']).values
    vuelos['TMIN_d'] = vuelosSinTratar.set_index(['index'])['TMIN_d'].combine_first(vuelos.set_index(['index'])['TMIN_d']).values
    vuelos['TMAX_o'] = vuelosSinTratar.set_index(['index'])['TMAX_o'].combine_first(vuelos.set_index(['index'])['TMAX_o']).values
    vuelos['TMAX_d'] = vuelosSinTratar.set_index(['index'])['TMAX_d'].combine_first(vuelos.set_index(['index'])['TMAX_d']).values
    vuelos['TAVG_o'] = vuelosSinTratar.set_index(['index'])['TAVG_o'].combine_first(vuelos.set_index(['index'])['TAVG_o']).values
    vuelos['TAVG_d'] = vuelosSinTratar.set_index(['index'])['TAVG_d'].combine_first(vuelos.set_index(['index'])['TAVG_d']).values
    vuelos['SNOW_o'] = vuelosSinTratar.set_index(['index'])['SNOW_o'].combine_first(vuelos.set_index(['index'])['SNOW_o']).values
    vuelos['SNOW_d'] = vuelosSinTratar.set_index(['index'])['SNOW_d'].combine_first(vuelos.set_index(['index'])['SNOW_d']).values
    vuelos['PRCP_o'] = vuelosSinTratar.set_index(['index'])['PRCP_o'].combine_first(vuelos.set_index(['index'])['PRCP_o']).values
    vuelos['PRCP_d'] = vuelosSinTratar.set_index(['index'])['PRCP_d'].combine_first(vuelos.set_index(['index'])['PRCP_d']).values
    vuelos['SNWD_o'] = vuelosSinTratar.set_index(['index'])['SNWD_o'].combine_first(vuelos.set_index(['index'])['SNWD_o']).values
    vuelos['SNWD_d'] = vuelosSinTratar.set_index(['index'])['SNWD_d'].combine_first(vuelos.set_index(['index'])['SNWD_d']).values
    vuelos['ACMC_o'] = vuelosSinTratar.set_index(['index'])['ACMC_o'].combine_first(vuelos.set_index(['index'])['ACMC_o']).values
    vuelos['ACMC_d'] = vuelosSinTratar.set_index(['index'])['ACMC_d'].combine_first(vuelos.set_index(['index'])['ACMC_d']).values
    vuelos['ACSC_o'] = vuelosSinTratar.set_index(['index'])['ACSC_o'].combine_first(vuelos.set_index(['index'])['ACSC_o']).values
    vuelos['ACSC_d'] = vuelosSinTratar.set_index(['index'])['ACSC_d'].combine_first(vuelos.set_index(['index'])['ACSC_d']).values
    vuelos['AWDR_o'] = vuelosSinTratar.set_index(['index'])['AWDR_o'].combine_first(vuelos.set_index(['index'])['AWDR_o']).values
    vuelos['AWDR_d'] = vuelosSinTratar.set_index(['index'])['AWDR_d'].combine_first(vuelos.set_index(['index'])['AWDR_d']).values
    vuelos['AWND_o'] = vuelosSinTratar.set_index(['index'])['AWND_o'].combine_first(vuelos.set_index(['index'])['AWND_o']).values
    vuelos['AWND_d'] = vuelosSinTratar.set_index(['index'])['AWND_d'].combine_first(vuelos.set_index(['index'])['AWND_d']).values
    vuelos['EVAP_o'] = vuelosSinTratar.set_index(['index'])['EVAP_o'].combine_first(vuelos.set_index(['index'])['EVAP_o']).values
    vuelos['EVAP_d'] = vuelosSinTratar.set_index(['index'])['EVAP_d'].combine_first(vuelos.set_index(['index'])['EVAP_d']).values
    vuelos['FRTH_o'] = vuelosSinTratar.set_index(['index'])['FRTH_o'].combine_first(vuelos.set_index(['index'])['FRTH_o']).values
    vuelos['FRTH_d'] = vuelosSinTratar.set_index(['index'])['FRTH_d'].combine_first(vuelos.set_index(['index'])['FRTH_d']).values
    vuelos['TSUN_o'] = vuelosSinTratar.set_index(['index'])['TSUN_o'].combine_first(vuelos.set_index(['index'])['TSUN_o']).values
    vuelos['TSUN_d'] = vuelosSinTratar.set_index(['index'])['TSUN_d'].combine_first(vuelos.set_index(['index'])['TSUN_d']).values
    vuelos['WDMV_o'] = vuelosSinTratar.set_index(['index'])['WDMV_o'].combine_first(vuelos.set_index(['index'])['WDMV_o']).values
    vuelos['WDMV_d'] = vuelosSinTratar.set_index(['index'])['WDMV_d'].combine_first(vuelos.set_index(['index'])['WDMV_d']).values

    
    #vuelos.to_csv('vuelos.csv', sep=',', index=False)
    end = time.time()
    print(end - start)

In [10]:
#variables de inicio
dia = 1
mes = 1
anyo = 2016

In [36]:
for mes in range(1,2):
    print 'empiezo el mes', mes
    mesw = str(mes)
    for dia in range(1,9):
        print 'empiezo el dia - ' ,dia
        diaw = str(dia)
        if dia < 10:
            diaw = '0'+diaw

        weatherdf = functionWeatherDay(anyo,mesw,diaw)
        functionPlus(anyo,mes,dia)

empiezo el mes 1
empiezo el dia -  1
3.76300001144
empiezo el dia -  2
3.62800002098
empiezo el dia -  3
3.58699989319
empiezo el dia -  4
3.64599990845
empiezo el dia -  5
3.62300014496
empiezo el dia -  6
3.56599998474
empiezo el dia -  7
3.53099989891
empiezo el dia -  8
4.32399988174


In [445]:
vuelos.to_csv('vuelos.csv', sep=',', index=False)


In [11]:
#para ver el estado del proceso...borrar luego.
vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
vuelosTratados = vuelos[vuelos['TAVG_o'].notnull()]
print 'sin tratar de todos ', vuelosSinTratar.shape
print 'tratados ', vuelosTratados.shape
x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==2012]
print 'sin tratar totales del año ', x.shape
x = x[x['mesSalida']==mes]
print 'del mes sin tratar ', x.shape
x = x[x['diaSalida']==dia]
print 'vuelos del dia sin tratar ' , x.shape

sin tratar de todos  (0, 86)
tratados  (212698, 86)
sin tratar totales del año  (0, 86)
del mes sin tratar  (0, 86)
vuelos del dia sin tratar  (0, 86)


In [10]:
vuelos.shape

(212698, 86)

In [11]:
vuelos[vuelos.TMIN_d==9999.0].shape

(93590, 86)

In [12]:
vuelos[vuelos.TMIN_o==9999.0].shape

(80587, 86)

In [13]:
vuelos[vuelos.TMAX_d==9999.0].shape

(80129, 86)

In [14]:
vuelos[vuelos.TMAX_o==9999.0].shape

(66248, 86)

In [15]:
vuelos[vuelos.TAVG_d==9999.0].shape

(50849, 86)

In [16]:
vuelos[vuelos.TAVG_o==9999.0].shape

(33124, 86)

In [17]:
vuelos[vuelos.SNOW_o==9999.0].shape

(212687, 86)

In [18]:
vuelos[vuelos.SNOW_d==9999.0].shape

(212691, 86)

In [19]:
vuelos[vuelos.PRCP_o==9999.0].shape

(45673, 86)

In [20]:
vuelos[vuelos.PRCP_d==9999.0].shape

(59212, 86)

In [21]:
vuelos[vuelos.SNWD_o==9999.0].shape

(209910, 86)

In [22]:
vuelos[vuelos.SNWD_d==9999.0].shape

(210240, 86)

In [23]:
vuelos[vuelos.ACMC_o==9999.0].shape

(212698, 86)

In [24]:
vuelos[vuelos.ACMC_d==9999.0].shape

(212698, 86)

In [25]:
vuelos[vuelos.ACSC_o==9999.0].shape

(212698, 86)

In [26]:
vuelos[vuelos.ACSC_d==9999.0].shape

(212698, 86)

In [27]:
vuelos[vuelos.AWDR_o==9999.0].shape

(212698, 86)

In [28]:
vuelos[vuelos.AWDR_d==9999.0].shape

(212698, 86)

In [29]:
vuelos[vuelos.AWND_o==9999.0].shape

(212687, 86)

In [30]:
vuelos[vuelos.AWND_d==9999.0].shape

(212691, 86)

In [31]:
vuelos[vuelos.EVAP_o==9999.0].shape

(212698, 86)

In [32]:
vuelos[vuelos.EVAP_d==9999.0].shape

(212698, 86)

In [33]:
vuelos[vuelos.FRTH_o==9999.0].shape

(212698, 86)

In [34]:
vuelos[vuelos.FRTH_d==9999.0].shape

(212698, 86)

In [35]:
vuelos[vuelos.TSUN_o==9999.0].shape

(212698, 86)

In [68]:
vuelos[vuelos.TSUN_d==9999.0].shape

(212698, 86)

In [69]:
vuelos[vuelos.WDMV_o==9999.0].shape

(212698, 86)

In [70]:
vuelos[vuelos.WDMV_d==9999.0].shape

(212698, 86)

In [10]:
#todas las estaciones
stationsdf.to_csv('stationsAll.csv',sep=',', index=False)

In [16]:
stationsTAVG =  weatherdf[weatherdf["type"]=="TAVG"]
stationsTAVG = stationsTAVG.sort_values(by=['id'])

In [13]:
def getLatstation(row):
    row['lng'] = stationsdf[stationsdf['id']==row['id']]['lng'].values[0]
    row['lat'] = stationsdf[stationsdf['id']==row['id']]['lat'].values[0]
    return row

In [14]:
vuelosSinTAVG_o = vuelos[vuelos.TAVG_o==9999.0]

In [15]:
vuelos[vuelos.TAVG_o==9999.0].shape

(12076, 86)

In [17]:
def obtenerTAVG_o(row):
    
    lat = row['board_lat']
    lon = row['board_lon']
    coor = [lat,lon]
    latsAux =  np.asarray(lats)
    EMC= closest(latsAux, coor)
    #PASAMOS A DATAFRAME Y BORRAMOS LA ESTACION DEL LISTADO.
    latsAux = pd.DataFrame(data = latsAux)
    latsAux = latsAux[(latsAux[0] != EMC[0]) | (latsAux[1] != EMC[1])]
    latsAux =  np.asarray(latsAux)
   
    flag = 'false'
    #filtramos por fecha el dataframe de staciones
    fechaAux = tratarFecha(row['actual_time_of_departure'])
    b = stationsTAVG[stationsTAVG.date == fechaAux]
    

    
    while(flag == 'false'):
        x = b
        EMC = closest(latsAux, coor)#BUSCAMOS LA 2a MAS CERCANA
       
        latsAux = pd.DataFrame(data = latsAux)
        
        latsAux = latsAux[(latsAux[0] != EMC[0]) | (latsAux[1] != EMC[1])]
        latsAux =  np.asarray(latsAux)
        
        ##obtenemos el id de la estacion de board
        aux = stationsdf[stationsdf['lat']==EMC[0]]
        aux = stationsdf[stationsdf['lng']==EMC[1]]
        EMC = aux['id']
        s =EMC.to_string()
        stationid = str(s)
        row['stationidTAVG_o'] = stationid.split()[1]
        
        x= x[x.id == stationid.split()[1]]
        
        if x.empty:
            pass
        else:    
            row['TAVG_o'] = float(x.Value1.values[0])/10
            flag = 'true'
    del b
    del x
    
    return row

In [19]:
def functionGest(year, month, day):
    start = time.time()
    vuelosSinTratar = vuelos[vuelos.TAVG_o==9999.0]
     
    x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==year]
    x = x[x['mesSalida']==month]
    x = x[x['diaSalida']==day]


    x = x.apply(lambda x:obtenerTAVG_o(x),axis = 1)

    pd.options.mode.chained_assignment = None  # default='warn'
    vuelosSinTratar['TAVG_o'] = x.set_index(['index'])['TAVG_o'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_o']).values
    
    vuelos['TAVG_o'] = vuelosSinTratar.set_index(['index'])['TAVG_o'].combine_first(vuelos.set_index(['index'])['TAVG_o']).values

    
    #vuelos.to_csv('vuelos.csv', sep=',', index=False)
    end = time.time()
    print(end - start)

In [17]:
anyo = 2016


for mes in range(1,7):
    print 'empiezo el mes', mes
    mesw = str(mes)
    for dia in range(1,32):
        print 'empiezo el dia - ' ,dia
        diaw = str(dia)
        if dia < 10:
            diaw = '0'+diaw

        #weatherdf = fu(anyo,mesw,diaw)
        functionGest(anyo,mes,dia)

empiezo el mes 1
empiezo el dia -  1
0.535000085831
empiezo el dia -  2
0.197000026703
empiezo el dia -  3
0.192000150681
empiezo el dia -  4
0.207999944687
empiezo el dia -  5
0.190999984741
empiezo el dia -  6
0.189999818802
empiezo el dia -  7
0.183000087738
empiezo el dia -  8
0.192999839783
empiezo el dia -  9
0.212000131607
empiezo el dia -  10
0.19000005722
empiezo el dia -  11
0.181999921799
empiezo el dia -  12
0.18799996376
empiezo el dia -  13
0.18499994278
empiezo el dia -  14
0.186000108719
empiezo el dia -  15
0.217000007629
empiezo el dia -  16
0.18799996376
empiezo el dia -  17
0.191999912262
empiezo el dia -  18
0.194000005722
empiezo el dia -  19
0.200999975204
empiezo el dia -  20
0.213999986649
empiezo el dia -  21
0.246999979019
empiezo el dia -  22
0.219000101089
empiezo el dia -  23
0.202000141144
empiezo el dia -  24
0.200000047684
empiezo el dia -  25
0.200999975204
empiezo el dia -  26
0.183000087738
empiezo el dia -  27
0.236000061035
empiezo el dia -  28
0.2

In [ ]:
vuelos.to_csv('vuelos.csv', sep=',', index=False)
vuelosSinTAVG_o = vuelos[vuelos.TAVG_o==9999.0]
vuelosSinTAVG_o.shape

(21652, 86)

In [21]:
vuelos[(vuelos.TMAX_o!=9999.0 )&(vuelos.TMAX_d!=9999.0 )&(vuelos.PRCP_o!=9999.0 )&(vuelos.PRCP_d!=9999.0 )&(vuelos.TMIN_o!=9999.0 )&(vuelos.TMIN_d!=9999.0 )]

,index,airline_code,flight_number,board_point,board_lat,board_lon,board_country_code,off_point,off_lat,off_lon,off_country_code,distance,scheduled_time_of_departure,actual_time_of_departure,scheduled_time_of_arrival,actual_time_of_arrival,departure_delay,arrival_delay,sched_blocktime,est_blocktime,act_blocktime,aircraft_type,aircraft_registration_number,general_status_code,routing,cabin_1_code,cabin_1_fitted_configuration,cabin_1_saleable,cabin_1_pax_boarded,cabin_1_rpk,cabin_1_ask,cabin_2_code,cabin_2_fitted_configuration,cabin_2_saleable,cabin_2_pax_boarded,cabin_2_rpk,cabin_2_ask,total_rpk,total_ask,load_factor,total_pax,total_no_shows,total_cabin_crew,total_technical_crew,total_baggage_weight,number_of_baggage_pieces,mesSalida,anyoSalida,diaSalida,horaSalida,mesLlegada,anyoLlegada,diaLlegada,horaLlegada,diaSemanaSalida,diaSemanaLlegada,TMIN_o,TMIN_d,TMAX_o,TMAX_d,TAVG_o,TAVG_d,SNOW_o,SNOW_d,PRCP_o,PRCP_d,SNWD_o,SNWD_d,ACMC_o,ACMC_d,ACSC_o,ACSC_d,AWDR_o,AWDR_d,AWND_o,AWND_d,EVAP_o,EVAP_d,FRTH_o,FRTH_d,TSUN_o,TSUN_d,WDMV_o,WDMV_d,board_stationid,off_stationid
163,165,XX,4756,LEU,42.338611,1.409167,ES,SPC,28.683510,-17.764210,ES,2296,2016-10-22 16:15:00,2016-10-22 16:15:00,2016-10-22 19:45:00,2016-10-22 19:52:00,0,7,270,NaN,277,321,XLMAT,Departed,LEU-SPC,Y,196,196,180,413275,450011,NaN,NaN,NaN,NaN,NaN,NaN,413275,450011,0.918367,180,0,4,2,2699,181,10,2016,22,16:15:00,10,2016,22,19:52:00,s�bado,s�bado,1.8,20.2,19.6,26.8,9.7,22.8,9999.0,9999.0,0.0,1.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,FRM00007627,SPE00120449
184,187,XX,6556,LEU,42.338611,1.409167,ES,SPC,28.683510,-17.764210,ES,2296,2016-10-22 16:30:00,2016-10-22 16:30:00,2016-10-22 20:00:00,2016-10-22 20:09:00,0,9,270,NaN,279,320,XJMAT,Departed,LEU-SPC,Y,165,165,160,367356,378836,NaN,NaN,NaN,NaN,NaN,NaN,367356,378836,0.969697,160,0,4,2,2378,158,10,2016,22,16:30:00,10,2016,22,20:09:00,s�bado,s�bado,1.8,20.2,19.6,26.8,9.7,22.8,9999.0,9999.0,0.0,1.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,FRM00007627,SPE00120449
222,225,XX,8556,LEU,42.338611,1.409167,ES,VDE,27.806280,-17.915780,ES,2378,2016-09-24 16:15:00,2016-09-24 16:19:00,2016-09-24 19:35:00,2016-09-24 19:32:00,4,-3,260,NaN,253,321,XLQAT,Departed,LEU-VDE,Y,196,196,180,427971,466013,NaN,NaN,NaN,NaN,NaN,NaN,427971,466013,0.918367,180,0,4,2,2424,171,9,2016,24,16:19:00,9,2016,24,19:32:00,s�bado,s�bado,9.7,23.2,27.0,26.2,18.2,23.8,9999.0,9999.0,0.0,0.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,FRM00007627,SPE00120431
243,246,XX,8356,LEU,42.338611,1.409167,ES,OUA,12.365660,-1.533880,BF,3345,2016-05-14 15:00:00,2016-05-14 15:01:00,2016-05-14 18:40:00,2016-05-14 18:40:00,1,0,340,NaN,339,75W,XNDAT,Departed,LEU-OUA,Y,227,227,175,585369,759307,NaN,NaN,NaN,NaN,NaN,NaN,585369,759307,0.770925,175,0,5,2,2589,169,5,2016,14,15:01:00,5,2016,14,18:40:00,s�bado,s�bado,10.5,23.0,14.3,36.2,11.9,28.9,9999.0,9999.0,2.0,9.9,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,FRM00007627,UVM00065503
248,251,XX,9556,LEU,42.338611,1.409167,ES,OUA,12.365660,-1.533880,BF,3345,2016-05-14 14:50:00,2016-05-14 14:55:00,2016-05-14 18:30:00,2016-05-14 18:32:00,5,2,340,NaN,337,319,XHMAT,Departed,LEU-OUA,Y,138,138,119,398051,461605,NaN,NaN,NaN,NaN,NaN,NaN,398051,461605,0.862319,119,0,3,2,1730,113,5,2016,14,14:55:00,5,2016,14,18:32:00,s�bado,s�bado,10.5,23.0,14.3,36.2,11.9,28.9,9999.0,9999.0,2.0,9.9,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,FRM00007627,UVM00065503
264,267,XX,556,LEU,42.338611,1.409167,ES,MZH,40.829375,35.521992,TR,2823,2016-04-09 14:50:00,2016-04-09 15:02:00,2016-04-09 20:50:00,2016-04-09 21:02:00,12,12,300,NaN,300,75W,XNEAT,Departed,LEU-

In [22]:
def getLatstation(row):
    row['lng'] = stationsdf[stationsdf['id']==row['id']]['lng'].values[0]
    row['lat'] = stationsdf[stationsdf['id']==row['id']]['lat'].values[0]
    return row


In [35]:
if os.path.exists('listaTAVG.csv')=='false':
    lista = stationsTAVG['id'].unique()
    lista = pd.DataFrame(lista)
    lista.columns = ['id']
    lista = lista.apply(lambda x:getLatstation(x),axis=1)
    lista.to_csv('listaTAVG.csv', sep=',', index=False)
else:
    lista = pd.read_csv('listaTAVG.csv',sep = ',')

In [88]:
def applylats(row):
    board_lat = row['board_lat']
    board_lon = row['board_lon']
    ------------------------------------mirar 
    
     = lista[lista['id']==row['id']]['lat'].values[0]
    
    row['estacionTAVG_o'] = station
    
    return row

In [89]:
stationsTAVG = stationsTAVG.apply(lambda x: applylats(x),axis = 1)

KeyboardInterrupt: 

In [43]:
lista

,id,lng,lat
0,AE000041196,55.5170,25.3330
1,AEM00041194,55.3640,25.2550
2,AEM00041217,54.6510,24.4330
3,AEM00041218,55.6090,24.2620
4,AFM00040938,62.2280,34.2100
5,AFM00040948,69.2120,34.5660
6,AFM00040990,65.8500,31.5000
7,AG000060390,3.2500,36.7167
8,AG000060590,2.8667,30.5667
9,AG000060611,9.6331,28.0500


In [40]:
vuelosSinTratar.sample(3)

,index,airline_code,flight_number,board_point,board_lat,board_lon,board_country_code,off_point,off_lat,off_lon,off_country_code,distance,scheduled_time_of_departure,actual_time_of_departure,scheduled_time_of_arrival,actual_time_of_arrival,departure_delay,arrival_delay,sched_blocktime,est_blocktime,act_blocktime,aircraft_type,aircraft_registration_number,general_status_code,routing,cabin_1_code,cabin_1_fitted_configuration,cabin_1_saleable,cabin_1_pax_boarded,cabin_1_rpk,cabin_1_ask,cabin_2_code,cabin_2_fitted_configuration,cabin_2_saleable,cabin_2_pax_boarded,cabin_2_rpk,cabin_2_ask,total_rpk,total_ask,load_factor,total_pax,total_no_shows,total_cabin_crew,total_technical_crew,total_baggage_weight,number_of_baggage_pieces,mesSalida,anyoSalida,diaSalida,horaSalida,mesLlegada,anyoLlegada,diaLlegada,horaLlegada,diaSemanaSalida,diaSemanaLlegada,TMIN_o,TMIN_d,TMAX_o,TMAX_d,TAVG_o,TAVG_d,SNOW_o,SNOW_d,PRCP_o,PRCP_d,SNWD_o,SNWD_d,ACMC_o,ACMC_d,ACSC_o,ACSC_d,AWDR_o,AWDR_d,AWND_o,AWND_d,EVAP_o,EVAP_d,FRTH_o,FRTH_d,TSUN_o,TSUN_d,WDMV_o,WDMV_d,board_stationid,off_stationid
197787,205515,XX,82,SUJ,47.799260,22.862550,RO,LEU,42.338611,1.409167,ES,1784,2015-05-24 20:10:00,2015-05-24 20:09:00,2015-05-24 20:50:00,2015-05-24 20:53:00,-1,3,100,NaN,104,319,XHSAT,Departed,SUJ-LEU,Y,122,104,102,181945,185512,J,1.0,15.0,10.0,17838.0,26757.0,199783,212269,0.941176,112,3,3,2,1339,86,5,2015,24,20:09:00,5,2015,24,20:53:00,domingo,domingo,9999.0,10.9,9999.0,20.7,9999.0,14.6,9999.0,9999.0,9999.0,4.6,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,FIE00144337,FRM00007627
177941,184655,QN,6999,LEU,42.338611,1.409167,ES,LIG,45.831530,1.257800,FR,389,2016-11-11 11:50:00,2016-11-11 11:47:00,2016-11-11 13:55:00,2016-11-11 14:05:00,-3,10,125,NaN,138,AT7,MFQAT,Departed,LEU-LIG,Y,68,68,29,11269,26424,NaN,NaN,NaN,NaN,NaN,NaN,11269,26424,0.426471,29,3,2,2,126,10,11,2016,11,11:47:00,11,2016,11,14:05:00,viernes,viernes,9999.0,9999.0,9999.0,9999.0,9999.0,5.0,9999.0,9999.0,9999.0,0.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,FRM00007627,FRM00007434
10310,10651,XX,5865,GRO,41.983110,2.824930,ES,LEU,42.338611,1.409167,ES,123,2017-03-20 16:25:00,2017-03-20 16:18:00,2017-03-20 18:20:00,2017-03-20 18:05:00,-7,-15,115,NaN,107,AT7,MFXAT,Departed,GRO-LEU,Y,68,68,32,3943,8378,NaN,NaN,NaN,NaN,NaN,NaN,3943,8378,0.470588,32,0,2,2,275,19,3,2017,20,16:18:00,3,2017,20,18:05:00,lunes,lunes,9999.0,6.9,9999.0,13.6,9999.0,9.7,9999.0,9999.0,9999.0,0.3,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,SPE00120071,FRM00007627
